# Mosaic Creation
With this notebook you will learn how to create a mosaic in form of a picture from hundreds of other pictures. This examplary notebooks works with a [flower dataset](https://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html). Some parts of this code is used from the project [mosaic-cv](https://github.com/nekrotzar/mosaic-cv).


In [ ]:
import os
import time 
import argparse

import cv2 as cv
import numpy as np
import progress.bar
from tqdm.notebook import tqdm, trange
from matplotlib import pyplot as plt

IMAGE_FILE_FORMATS = ['.JPG', '.jpg', '.png']

#images_path = './data_flowers/'
#target_path = './data/Porsche_Taycan.jpg'
images_path = 'C:/Users/info/OneDrive/Pictures/'
target_path = 'data/_DSC0914.JPG'
target_scale = 10
block_size = 100

In [ ]:
def process_image(path, size):
    # crop and resize images according to given block size
    # path: image directory containing mosaic images
    # size: size of mosaic block (pixels)
    # returns np.ndarray containing BGR color values
    
    image = cv.imread(path)
    height, width = image.shape[0], image.shape[1]

    # Crop image in order to avoid distortion
    min_dim = np.min((height, width))
    w_crop = int((width-min_dim) / 2)
    h_crop = int((height-min_dim) / 2)
    crop = image[h_crop:height-h_crop, w_crop:width-w_crop]

    # Resize cropped image to target block size
    target_image = cv.resize(crop, (size, size))
    return target_image

data = []
print("Analysing dataset...")
for (root, _, filenames) in os.walk(images_path):       
    for filename in filenames:
        if filename[-4:] in IMAGE_FILE_FORMATS:
            path = os.path.join(root, filename)
            proc = process_image(path, block_size)
            data.append(proc)

In [ ]:
# Read target image
target = cv.imread(target_path)
target = cv.resize(target, None, fx=target_scale, fy=target_scale, interpolation=cv.INTER_AREA)
height, width = target.shape[:2]

# Crop image to fit an integer number of mosaic blocks per row and column
w_crop = int((width % block_size))
h_crop = int((height % block_size))

if w_crop or h_crop:
    w_adjust = 1 if (w_crop % 2) else 0
    h_adjust = 1 if (h_crop % 2) else 0
    min_h, min_w = int(h_crop/2), int(w_crop/2)
    target = target[min_h:height-(min_h + h_adjust), min_w: width-(min_w + w_adjust)]

# Create blank image with target dimensions
output = np.zeros(target.shape, dtype=np.uint8)
(height, width) = output.shape[:2]

row_size = int(height / block_size)
col_size = int(width / block_size)
n_blocks = row_size * col_size
print("\tNumber of mosaic blocks: {0} rows x {1} cols = {2}".format(row_size, col_size, n_blocks)) 

In [ ]:
def average_color(data):
    # get average color 

    # data: np.ndarray, an array of array of length 3 (R,G,B)
    average_row_color = np.average(data, axis=0)
    average_color = np.average(average_row_color, axis=0)

    return average_color

plt.imshow(data[0])
plt.show()

# get average color
avg_color = average_color(data[0])
print('Average color: ', avg_color)
img_color_r = np.ndarray([10,10])
img_color_g = np.ndarray([10,10])
img_color_b = np.ndarray([10,10])
img_color_r = np.full((10,10), int(avg_color[0]))
img_color_g = np.full((10,10), int(avg_color[1]))
img_color_b = np.full((10,10), int(avg_color[2]))
img_color_rgb = np.dstack((img_color_r,img_color_g,img_color_b))
plt.imshow(img_color_rgb)
plt.show()

In [ ]:
# calculate average color for every input image
data_avg_color = np.ndarray((len(data),3))
n_blocks = len(data)
print(n_blocks)
#pbar = progress.bar.IncrementalBar('Create average colors...', max=n_blocks)
for i in trange(len(data)):
    data_avg_color[i] = average_color(data[i])
    #pbar.next()

#pbar.finish()
# convert to uint
data_avg_color = np.uint8(data_avg_color)


In [ ]:
# find best fit from dataset
def find_best_match(block, target_value, data):

    
    # block: block from target
    # target_value: target metric value (RGB color array)
    # data: blocks processed from the dataset

    # returns numpy.ndarray: a block that best matches the color    
    min_distance = np.Infinity
    best_match = None

    for i in range(len(data)):    
        color = average_color(data[i])
        distance = np.linalg.norm(target_value - color)

        if distance < min_distance:
            min_distance = distance
            best_match = i

    return best_match

#pbar = progress.bar.IncrementalBar('Building mosaic', max=n_blocks)
for r in range(row_size):
    print('Processing row ', r)
    for c in range(col_size):
        h_pos = (block_size * r)
        w_pos = (block_size * c)

        block = target[h_pos:h_pos+block_size, w_pos:w_pos+block_size]

        # Use average color as metric for block similarity
        metric_value = average_color(block)
        block = data[find_best_match(block, metric_value, data)]
        
        # Fill output with average color
        output[h_pos:h_pos+block_size, w_pos:w_pos+block_size] = block
        #pbar.next()
    
#pbar.finish()

In [ ]:
cv.imwrite('mosaic_flowers.png', output)